# Train A Shape Classifier Model



In [ ]:
import json
import os

train_data_root = "../datasets/train"
test_data_root = "../datasets/test"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations (including resizing and normalization)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale (black and white images)
    transforms.Resize((64, 64)),  # Resize images to 64x64 pixels
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images (mean=0.5, std=0.5 for grayscale)
])

# Load the dataset
train_dataset = datasets.ImageFolder(root=train_data_root, transform=transform)
test_dataset = datasets.ImageFolder(root=test_data_root, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check class names (optional)
print(f'Classes: {train_dataset.classes}')

# 2. Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1) #tensor shape: (batch_size, 1, 64, 64) to tensor shape: (batch_size, 16, 32, 32) due to max pooling
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1) #tensor shape: (batch_size, 32, 32, 32) to tensor shape: (batch_size, 32, 16, 16) due to max pooling
        self.fc1 = nn.Linear(32 * 16 * 16, 128) # 32*16*16 are the amount of features
        self.fc2 = nn.Linear(128, 3)  # 3 classes: circle, triangle, rectangle
        
    def forward(self, x):
        x = F.relu(self.conv1(x))   # First Conv Layer
        x = F.max_pool2d(x, 2)      # Max Pooling
        x = F.relu(self.conv2(x))   # Second Conv Layer
        x = F.max_pool2d(x, 2)      # Max Pooling
        x = x.view(x.size(0), -1)   # Flatten
        x = F.relu(self.fc1(x))     # Fully Connected Layer 1
        x = self.fc2(x)             # Fully Connected Layer 2 (output)
        return x

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%')


# Train

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print image shape before training
images, labels = next(iter(train_loader))
print(f'Image batch shape: {images.shape}')




In [ ]:
train_model(model, train_loader, criterion, optimizer, epochs=15)

In [ ]:
print(f"Test dataset size: {len(test_loader.dataset)}")

# Test 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
import numpy as np

def test(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Calculate precision, recall and f1-score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average=None)
    
    def safe_iter(x):
        # If scalar, convert to 1-element list for iteration
        if np.isscalar(x):
            return [x]
        return x

    precision = safe_iter(precision)
    recall = safe_iter(recall)
    f1 = safe_iter(f1)

    for i, (p, r, f) in enumerate(zip(precision, recall, f1)):
        print(f'Class {i}, Precision: {p:.4f}, Recall: {r:.4f}, F1-score: {f:.4f}')

    # Calculate and print overall (macro) averages
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
    print(f'Overall (Macro) Precision={precision_macro:.4f}, Recall={recall_macro:.4f}, F1-Score={f1_macro:.4f}')

# Call test function after training
test(model, test_loader)


# Show Predictions


In [ ]:
import matplotlib.pyplot as plt
import torch.nn.functional as F

# Class names matching your dataset folders order
class_names = ['circle', 'diamond', 'triangle']

def show_prediction(model, image):
    model.eval()

    # Add batch dimension if missing
    if image.ndim == 3:
        image = image.unsqueeze(0)  # shape [1, C, H, W]

    with torch.no_grad():
        outputs = model(image.to(device))
        probs = F.softmax(outputs, dim=1)  # Convert logits to probabilities
        conf, pred_idx = torch.max(probs, 1)

    # Convert tensor to CPU numpy array for plotting
    img_np = image.squeeze().cpu().numpy()

    # Plot image
    plt.imshow(img_np, cmap='gray')
    plt.axis('off')

    # Prepare text with prediction and confidence %
    label = class_names[pred_idx.item()]
    confidence = conf.item() * 100
    title = f'Prediction: {label} ({confidence:.2f}%)'

    plt.title(title)
    plt.show()

# Example usage: show a prediction for the 1st image in the test dataset
image, label = test_dataset[0]
show_prediction(model, image)
